#  Queries and Fetches

A database **query** is a formal question posed to a collection of stored data.
Queries can be understood as functions on stored data to present a precise cross-section, tailored rigorously and efficiently for the analysis at hand.


In DataJoint, the process of retrieving data involves three fundamental concepts: **query expressions**, **queries**, and **fetched results**. Understanding these concepts and how they relate to each other is essential for effective data manipulation and retrieval.


A query language provides a universal, declarative method for specifying the desired result, leaving the complex procedural details of how to locate, retrieve, and combine the data to the database management system. Rather than loading entire datasets into memory and then filtering them (as you might with files), queries allow you to specify *exactly* what data you need, and only that data is retrieved from the database.

### Queries vs. File-Based Data Retrieval

If you're new to databases, it helps to contrast queries with the familiar process of working with files and folders:

**File-Based Approach:**
```python
# Traditional file-based workflow
import pandas as pd

# 1. Load entire CSV file into memory
all_animals = pd.read_csv('animals.csv')  # Loads ALL data

# 2. Filter in Python
young_dogs = all_animals[(all_animals['age'] < 5) & 
                          (all_animals['species'] == 'Dog')]

# Problem: Must load all data first, even if you only need a few rows
```

**Database Query Approach:**
```python
# DataJoint workflow
import datajoint as dj

# 1. Define what you want (no data loaded yet)
young_dogs = Animal & 'age < 5' & 'species="Dog"'

# 2. Fetch only the specific data you need
result = young_dogs.fetch()

# Advantage: Only the relevant data is retrieved from the database
```

**Key Differences:**

| Aspect | Files & Folders | Database Queries |
|--------|----------------|------------------|
| **Data Loading** | Must load entire files | Retrieve only needed data |
| **Memory Usage** | All data in memory | Only results in memory |
| **Speed** | Slow for large files | Fast, even with huge datasets |
| **Filtering** | Done in Python after loading | Done by database before retrieval |
| **Relationships** | Manual (e.g., matching filenames) | Built-in (foreign keys, joins) |
| **Consistency** | Risk of version conflicts | Single source of truth |

### Why Queries Matter

Queries are powerful because they:
1. **Minimize data transfer**: Only retrieve what you need
2. **Reduce memory footprint**: Don't load unnecessary data
3. **Leverage database optimization**: Databases are highly optimized for searching and filtering
4. **Enable composition**: Build complex queries from simple building blocks
5. **Maintain relationships**: Automatically handle connections between related data

## Query Expressions, Queries, and Fetched Results

### Query Expressions

A **query expression** is a symbolic representation of a dataset within your DataJoint pipeline. Query expressions are composable — you can combine them using various operators to create more complex expressions. Importantly, query expressions are **lazy**: they don't actually retrieve data from the database until you explicitly request it.

The simplest query expression is **an entire table**. For example, if you have a table class `Animal`, then `Animal` itself is a query expression representing all rows in that table.

```python
import datajoint as dj

schema = dj.Schema('example_schema')

@schema
class Animal(dj.Manual):
    definition = """
    animal_id: int  # Unique identifier for the animal
    ---
    species: varchar(64)  # Species of the animal
    age: int             # Age of the animal in years
    """

# Animal is a query expression representing the entire table
query_expr = Animal()
```

### From Query Expressions to Queries

When you work with query expressions (like `Animal & 'age < 5'`), DataJoint constructs an underlying **query** — the actual execution plan for retrieving data from the database. This query is typically expressed as SQL, but DataJoint abstracts this away so you can work in pure Python.

The key insight: query expressions describe *what* data you want, and DataJoint figures out *how* to get it efficiently.

### Fetched Results

**Fetched results** are the actual data retrieved from the database after executing a query. You obtain fetched results by calling `fetch()` or `fetch1()` methods on a query expression. At this point, DataJoint communicates with the database, executes the SQL query, and returns the results in a format you specify (tuples, dictionaries, NumPy arrays, etc.).

```python
# Query expression (lazy, no database access yet)
young_animals = Animal & 'age < 3'

# Fetched result (database is queried, data is retrieved)
result = young_animals.fetch()
```

### The Workflow: Expression → Query → Result

1. **Build** a query expression using tables and operators
2. DataJoint **constructs** the underlying query (SQL)
3. **Fetch** executes the query and retrieves results

## Query Operators and Algebraic Closure

DataJoint provides a rich set of **query operators** that allow you to compose complex query expressions from simpler ones. These operators implement relational algebra operations and include:

- **Restriction** (`&`, `-`): Filter rows based on conditions
- **Join** (`*`): Combine tables based on common attributes
- **Projection** (`.proj()`): Select specific attributes or compute new ones
- **Aggregation** (`.aggr()`): Compute summary statistics
- **Union** (`+`): Combine results from multiple queries

### Algebraic Closure

A key property of DataJoint's query system is **algebraic closure**: when you apply operators to query expressions, the result is always another query expression. This means you can chain operations together indefinitely, building complex queries from simple building blocks.

```python
# Each operation returns a query expression
step1 = Animal  # query expression: entire table
step2 = Animal & 'age < 5'  # query expression: restricted table
step3 = (Animal & 'age < 5').proj('species')  # query expression: restricted and projected

# You can chain operations because of algebraic closure
complex_query = Animal & 'age < 5' & 'species="Dog"'
```

This composability is powerful because:
1. You can build complex queries incrementally
2. You can store intermediate query expressions as variables
3. All query expressions share the same interface (they support the same operations)
4. Operations remain lazy until you call `fetch()`

## Overview of `fetch`

The `fetch` command is designed to retrieve multiple rows of data or an entire result set from a table. It is highly versatile, allowing users to extract data as a list, NumPy array, or even as a pandas DataFrame for further analysis.

### Syntax

```python
<Table>.fetch(*attributes, as_dict=False, as_numpy=False, squeeze=False, order_by=None)
```

### Parameters

1. **`*attributes`** *(optional)*:
   - Specifies the attributes to fetch. If omitted, all attributes are retrieved.
2. **`as_dict`** *(default: False)*:
   - If `True`, the result is returned as a list of dictionaries.
3. **`as_numpy`** *(default: False)*:
   - If `True`, the result is returned as a NumPy array.
4. **`squeeze`** *(default: False)*:
   - If `True`, simplifies the result by removing redundant dimensions when a single attribute is fetched.
5. **`order_by`** *(optional)*:
   - Specifies the order of rows in the result set.

### Example

```python
import datajoint as dj

schema = dj.Schema('example_schema')

@schema
class Animal(dj.Manual):
    definition = """
    animal_id: int  # Unique identifier for the animal
    ---
    species: varchar(64)  # Species of the animal
    age: int             # Age of the animal in years
    """

# Fetch all rows as dictionaries
all_animals = Animal.fetch(as_dict=True)

# Fetch specific attributes
species = Animal.fetch('species')

# Fetch with ordering
ordered_animals = Animal.fetch(order_by='age')
```

### Key Points
- Use `fetch` to retrieve multiple rows or entire result sets.
- Flexible output formats (`dict`, `NumPy`, or default tuples) make it adaptable to various workflows.
- Supports attribute selection and row ordering for precise queries.

## Overview of `fetch1`

The `fetch1` command is used to retrieve a single row of data. It is ideal when querying tables with a single result or when the user is certain the query will yield exactly one row. Unlike `fetch`, `fetch1` raises an error if the query returns multiple rows or no rows at all.

### Syntax

```python
<Table>.fetch1(*attributes, squeeze=False)
```

### Parameters

1. **`*attributes`** *(optional)*:
   - Specifies the attributes to fetch. If omitted, all attributes are retrieved.
2. **`squeeze`** *(default: False)*:
   - If `True`, simplifies the result by removing redundant dimensions when a single attribute is fetched.

### Example

```python
# Insert some example data
Animal.insert1({
    'animal_id': 1, 'species': 'Dog', 'age': 5
})

# Fetch a single row
single_animal = Animal.fetch1(as_dict=True)

# Fetch a single attribute
species = Animal.fetch1('species')
```

### Key Points
- `fetch1` ensures exactly one result is returned, making it safer for single-row queries.
- Raises an error if the query yields zero or multiple results, enforcing strict query expectations.

## Comparison of `fetch` and `fetch1`

| Feature            | `fetch`                       | `fetch1`                     |
|--------------------|-------------------------------|------------------------------|
| **Rows Retrieved** | Multiple rows or entire set   | Exactly one row              |
| **Output Formats** | Tuples (default), dict, NumPy | Tuple (default), simplified  |
| **Error Handling** | No errors on empty results    | Errors on zero/multiple rows |
| **Use Case**       | Batch data retrieval          | Single-row data retrieval    |

## Best Practices

1. **Choose Based on Query Expectations**:
   - Use `fetch1` only when you are confident the query returns exactly one result.
   - Use `fetch` for multi-row queries or when unsure about the result count.
2. **Optimize Output Format**:
   - Use `as_dict=True` for user-friendly data exploration.
   - Use `as_numpy=True` for numerical computations.
3. **Order Your Queries**:
   - Leverage the `order_by` parameter in `fetch` to control row ordering.
4. **Test Your Queries**:
   - Test with `fetch` first to verify the result set before switching to `fetch1`.

## Summary

- `fetch` is ideal for retrieving multiple rows or entire datasets, offering flexible output formats.
- `fetch1` ensures strict control over single-row queries, making it perfect for exact matches.
- Both commands support attribute selection and efficient querying, enabling seamless data retrieval in your DataJoint pipeline.

